Домашняя работа 2   
Зенина Алёна БКЛ-201
# 1. Создание корпуса
В качестве корпуса я решила взять лингвистические анектдоты с сайта http://new.gramota.ru/biblio/gostinaya/lingvisticheskie-anekdoty, поскольку в них по идее должны встречаться различные сложные случаи: омонимия, необычные слова и другие приемы для достижения комического эффекта
### 1.1. Сбор текстов

In [1]:
import requests
from bs4 import BeautifulSoup

Собираю все анекдоты с сайта в массив

In [2]:
req = 'http://new.gramota.ru/biblio/gostinaya/lingvisticheskie-anekdoty'
response = requests.get(req)
texts = ''

if (response.status_code == 200):
    soup = BeautifulSoup(response.text, 'html.parser')
    dirty_texts = soup.findAll('div', {'itemprop': 'articleBody'})[0].findAll('p')
    
for text in dirty_texts:
    x = text.get_text().replace(u'\xa0', u' ')
    
    if x != ' ':
        texts += '\n' + x

In [241]:
import nltk
tokenizer = NLTKWordTokenizer()

def number_of_word(stroka):
    result = 0
    
    for word in tokenizer.tokenize(stroka.lower()):
    
        if word.strip('.,!?:…—–«»') != '':
            result += 1
            
    print('Слов всего ', result)

number_of_word(texts)

Слов всего  1472


Многовато слов, поэтому дальше я вручную выберу наиболее актуальные анекдоты

In [4]:
with open("тексты.txt", "w") as f:
    f.write(texts)

В целом идея с анекдотами оказалось более менее актуальной. Но для проверки POS-теггера необходимы скорее случаи омоформии, а не омонимии, которая встречается в собранных анекдотах намного чаще, поэтому я оставила только наиболее актуальные анекдоты и добавила детский стишок с омоформами

In [37]:
corpus_text = ''

with open("тексты готовые.txt", "r") as f:
    
    for line in f:
        corpus_text += line        

In [38]:
print(corpus_text)
number_of_word(corpus_text)

– Я вырос в то время, когда бутер еще был бутербродом.
Когда видишь алфавит, всё время ищешь ответ на вопрос: ГДЕ ЁЖ?
Только в русском языке можно составить предложение из трех гласных букв: – Э, а я?
Главпочтамту требуются специалисты по лизингу и маркетингу для наклеивания марок на конверты.
– Смотри: шампунь против выпадения волос. – Я тоже против, а что толку.
– Любимая, мне посуду мыть или ты вернeшься и сама помоешь? – Хорошо мой любимый... – Уточни, плиз: «Хорошо, мой любимый», «Хорошо, мой, любимый» или «Хорошо мой, любимый»?!
Парень зовeт девушку: «Женя! Женечка! Женюсь!» Девушка: «Ой! Уже иду!» 
– Как измерить силушку богатырскую? – Надо умножить массушку на ускореньице!
При раскопках индейского поселения Тау-Хау была обнаружена золотая статуэтка Кетцальмигонкуганькоальктенотчетлана – бога дикции и памяти.
Нажимаю «Мой компьютер», а он не моет! 
Розочка, а что это у нас сегодня на обед? – Картошка в депрессии. – Как это? – Ну, пюре. Вроде картошка как картошка, но такая подав

И слов как раз, и есть следующие интересные случаи:
* омоформы (мой, напасть)
* продуктивные модели от неожиданных слов (массушка, ускореньице)
* необычные слова (Кетцальмигонкуганькоальктенотчетлан, ийунь)
* слова с « типо неизвестными» корнями, но которые можно распознать по форме (прдаю, хршем)
* слова с дефисом внутри (IT-хаб, Тау-Хау)
### 1.2. Разметка

Перед разметкой возникают проблемы, так как у каждого анализатора свой набор тегов, поэтому я буду размечать по набору тегов, который предлагается в "Оценка методов автоматического анализа текста: морфологические парсеры русского языка", а именно:
* существительные (S)
* прилагательные (A)
* глаголы, причастия и деепричастия (V)
* предлоги (PR)
* союзы (CONJ)
* прочие несклоняемые слова: наречия, вводные слова, частицы, междометия, категории состояния (ADV)  
И к этому списку я еще добавила 
* числительные (NUM)
* местоимения (PRO)  
потому что используемые мной анализаторы их выделяют (конечно еще и делят между собой на категории, но это уже сильно много тегов получается). С составными предлогами и союзами нет проблем, потому что у меня в тексте они не встречаются. Затем выбранные теги я переведу в теги используемого анализатора     
Делаю разметку и записываю в json-файл

In [7]:
tagged_corpus = {}
token_corpus = tokenizer.tokenize(corpus.lower())
i = 0

for token in token_corpus:
    
    if token.strip('.,!?:…—–«»') != '':
        word = token.strip(".,!?:…—–«»")
        tags = tuple(input(f'{i}. POS of "{word}": ').split(' '))
        tagged_corpus[i] = {'word': word, 'tags': tags}
        i += 1

0. POS of "я": PRO
1. POS of "вырос": V
2. POS of "в": PR
3. POS of "то": PRO
4. POS of "время": S
5. POS of "когда": CONJ
6. POS of "бутер": S
7. POS of "еще": ADV
8. POS of "был": V
9. POS of "бутербродом": S
10. POS of "когда": CONJ
11. POS of "видишь": V
12. POS of "алфавит": S
13. POS of "всё": PRO
14. POS of "время": S
15. POS of "ищешь": V
16. POS of "ответ": S
17. POS of "на": PR
18. POS of "вопрос": S
19. POS of "где": ADV
20. POS of "ёж": S
21. POS of "только": ADV
22. POS of "в": PR
23. POS of "русском": A
24. POS of "языке": S
25. POS of "можно": ADV
26. POS of "составить": V
27. POS of "предложение": S
28. POS of "из": PR
29. POS of "трех": NUM
30. POS of "гласных": A
31. POS of "букв": S
32. POS of "э": ADV
33. POS of "а": CONJ
34. POS of "я": PRO
35. POS of "главпочтамту": S
36. POS of "требуются": V
37. POS of "специалисты": S
38. POS of "по": PR
39. POS of "лизингу": S
40. POS of "и": CONJ
41. POS of "маркетингу": S
42. POS of "для": PR
43. POS of "наклеивания": S
44. 

In [10]:
import json

In [11]:
with open('корпус.json', 'w', encoding='utf-8') as f:
    json.dump(tagged_corpus, f, ensure_ascii=False, indent='\t')

Скачиваю корпус обратно и превращаю текстовые значения индексов в числовые

In [32]:
with open('корпус.json', 'r', encoding='utf-8') as f:
    corpus = json.load(f)

for i in range(245): 
    corpus[i] = corpus[str(i)]
    corpus.pop(str(i))

corpus

{0: {'word': 'я', 'tags': ['PRO']},
 1: {'word': 'вырос', 'tags': ['V']},
 2: {'word': 'в', 'tags': ['PR']},
 3: {'word': 'то', 'tags': ['PRO']},
 4: {'word': 'время', 'tags': ['S']},
 5: {'word': 'когда', 'tags': ['CONJ']},
 6: {'word': 'бутер', 'tags': ['S']},
 7: {'word': 'еще', 'tags': ['ADV']},
 8: {'word': 'был', 'tags': ['V']},
 9: {'word': 'бутербродом', 'tags': ['S']},
 10: {'word': 'когда', 'tags': ['CONJ']},
 11: {'word': 'видишь', 'tags': ['V']},
 12: {'word': 'алфавит', 'tags': ['S']},
 13: {'word': 'всё', 'tags': ['PRO']},
 14: {'word': 'время', 'tags': ['S']},
 15: {'word': 'ищешь', 'tags': ['V']},
 16: {'word': 'ответ', 'tags': ['S']},
 17: {'word': 'на', 'tags': ['PR']},
 18: {'word': 'вопрос', 'tags': ['S']},
 19: {'word': 'где', 'tags': ['ADV']},
 20: {'word': 'ёж', 'tags': ['S']},
 21: {'word': 'только', 'tags': ['ADV']},
 22: {'word': 'в', 'tags': ['PR']},
 23: {'word': 'русском', 'tags': ['A']},
 24: {'word': 'языке', 'tags': ['S']},
 25: {'word': 'можно', 'tags':

# 2. Работа с анализаторами
Я использую mystem, pymorphy и natashеньку
### 2.1. Mystem
Создаю функцию-конвертер тегов

In [34]:
def mystem(tag):
    tagset = {
        'A': 'A',
        'ADV': 'ADV',
        'ADVPRO': 'ADV',
        'ANUM': 'NUM',
        'APRO': 'PRO',
        'COM': 'ADV',
        'CONJ': 'CONJ',
        'INTJ': 'ADV',
        'NUM': 'NUM',
        'PART': 'ADV',
        'PR': 'PR',
        'S': 'S',
        'SPRO': 'PRO',
        'V': 'V'
    }
    return tagset[tag]

'NUM'

Провожу анализ

In [166]:
from pymystem3 import Mystem
m = Mystem()

In [167]:
ana = m.analyze(corpus_text)

In [168]:
mystem_corpus = {}
i = 0

for word in ana:
    
    if 'analysis' in word:
        
        if word['analysis'] != []:
            pos = word['analysis'][0]['gr'].split('=')[0].split(',')[0]
            mystem_corpus[i] = {'word': word['text'], 'tag': mystem(pos)}
            
        else:
            mystem_corpus[i] = {'word': word['text'], 'tag': 'error'}
        i += 1

Особенность mystema заключается в том, что он всегда делит слова по дефису, а значит, придется чистить словарь от этого разбиения, часть речи оставляю по последней части слова (было Тау-Хау, будет Хау)

In [169]:
for i in range(106, 180):
    mystem_corpus[i] = mystem_corpus[i+1]
    
for i in range(180, 182):
    mystem_corpus[i] = mystem_corpus[i+2]
    
for i in range(182, 245):
    mystem_corpus[i] = mystem_corpus[i+4]

mystem_corpus.pop(245)
mystem_corpus.pop(246)
mystem_corpus.pop(247)
mystem_corpus.pop(248)

mystem_corpus

{0: {'word': 'Я', 'tag': 'PRO'},
 1: {'word': 'вырос', 'tag': 'V'},
 2: {'word': 'в', 'tag': 'PR'},
 3: {'word': 'то', 'tag': 'PRO'},
 4: {'word': 'время', 'tag': 'S'},
 5: {'word': 'когда', 'tag': 'CONJ'},
 6: {'word': 'бутер', 'tag': 'S'},
 7: {'word': 'еще', 'tag': 'ADV'},
 8: {'word': 'был', 'tag': 'V'},
 9: {'word': 'бутербродом', 'tag': 'S'},
 10: {'word': 'Когда', 'tag': 'CONJ'},
 11: {'word': 'видишь', 'tag': 'V'},
 12: {'word': 'алфавит', 'tag': 'S'},
 13: {'word': 'всё', 'tag': 'PRO'},
 14: {'word': 'время', 'tag': 'S'},
 15: {'word': 'ищешь', 'tag': 'V'},
 16: {'word': 'ответ', 'tag': 'S'},
 17: {'word': 'на', 'tag': 'PR'},
 18: {'word': 'вопрос', 'tag': 'S'},
 19: {'word': 'ГДЕ', 'tag': 'ADV'},
 20: {'word': 'ЁЖ', 'tag': 'S'},
 21: {'word': 'Только', 'tag': 'ADV'},
 22: {'word': 'в', 'tag': 'PR'},
 23: {'word': 'русском', 'tag': 'A'},
 24: {'word': 'языке', 'tag': 'S'},
 25: {'word': 'можно', 'tag': 'ADV'},
 26: {'word': 'составить', 'tag': 'V'},
 27: {'word': 'предложение'

Отлично, всё совпало!
### 2.2. Pymorphy
Опять функция-конвертер

In [102]:
def pymorphy(tag):
    tagset = {
        'NOUN': 'S',
        'ADJF': 'A',
        'ADJS': 'A',
        'COMP': 'A',
        'VERB': 'V',
        'INFN': 'V',
        'PRTF': 'V',
        'PRTS': 'V',
        'GRND': 'V',
        'NUMR': 'NUM',
        'ADVB': 'ADV',
        'NPRO': 'PRO',
        'PRED': 'ADV',
        'PREP': 'PR',
        'CONJ': 'CONJ',
        'PRCL': 'ADV',
        'INTJ': 'ADV'
    }
    return tagset[tag]    

Снова провожу анализ

In [103]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [105]:
pymorphy_corpus = {}
i = 0

for token in token_corpus:
    
        if token.strip('.,!?:…—–«»') != '':
            pos = morph.parse(token.strip('.,!?:…–«»'))[0].tag.POS
            
            if pos != None:
                pymorphy_corpus[i] = {'word': token.strip('.,!?:…–«»'), 'tag': pymorphy(pos)}
                
            else:
                pymorphy_corpus[i] = {'word': token.strip('.,!?:…–«»'), 'tag': 'error'}
                
            i += 1

pymorphy_corpus

Поправлять ничего не надо, потому что pymorphy принимает уже токенизированный заранее текст, а mystem делает это сам во время анализа.
### 2.3. Natasha
И снова функция-конвертер

In [107]:
def natasha(tag):
    tagset = {
        'ADJ': 'A',
        'ADV': 'ADV',
        'INTJ': 'ADV',
        'NOUN': 'S',
        'PROPN': 'S',
        'VERB': 'V',
        'ADP': 'PR',
        'AUX': 'V',
        'CCONJ': 'CONJ',
        'DET': 'PRO',
        'NUM': 'NUM',
        'PART': 'ADV',
        'PRON': 'PRO',
        'PUNCT': 'ADV',
        'SCONJ': 'CONJ',
        'SYM': 'ADV',
        'X': 'error'
    }
    return tagset[tag] 

Анализ Наташей

In [158]:
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, Doc
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(corpus_text)

In [159]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [179]:
natasha_corpus = {}
i = 0

for token in doc.tokens:
    
    if token.pos != 'PUNCT':
        natasha_corpus[i] = {'word': token.text, 'tag': natasha(token.pos)}
        i += 1

natasha_corpus

{0: {'word': 'Я', 'tag': 'PRO'},
 1: {'word': 'вырос', 'tag': 'V'},
 2: {'word': 'в', 'tag': 'PR'},
 3: {'word': 'то', 'tag': 'PRO'},
 4: {'word': 'время', 'tag': 'S'},
 5: {'word': 'когда', 'tag': 'CONJ'},
 6: {'word': 'бутер', 'tag': 'S'},
 7: {'word': 'еще', 'tag': 'ADV'},
 8: {'word': 'был', 'tag': 'V'},
 9: {'word': 'бутербродом', 'tag': 'V'},
 10: {'word': 'Когда', 'tag': 'CONJ'},
 11: {'word': 'видишь', 'tag': 'V'},
 12: {'word': 'алфавит', 'tag': 'S'},
 13: {'word': 'всё', 'tag': 'PRO'},
 14: {'word': 'время', 'tag': 'S'},
 15: {'word': 'ищешь', 'tag': 'V'},
 16: {'word': 'ответ', 'tag': 'S'},
 17: {'word': 'на', 'tag': 'PR'},
 18: {'word': 'вопрос', 'tag': 'S'},
 19: {'word': 'ГДЕ', 'tag': 'S'},
 20: {'word': 'ЁЖ', 'tag': 'ADV'},
 21: {'word': 'Только', 'tag': 'ADV'},
 22: {'word': 'в', 'tag': 'PR'},
 23: {'word': 'русском', 'tag': 'A'},
 24: {'word': 'языке', 'tag': 'S'},
 25: {'word': 'можно', 'tag': 'ADV'},
 26: {'word': 'составить', 'tag': 'V'},
 27: {'word': 'предложение'

Наташа сошла с ума и поделила мне слова *вернешься* и *зовет* на 3 слова, причем, если смотреть по предложениям, там все нормально, я думаю это как-то связано с ё(( В итоге я просто уберу первые два кусочка, и оба разделенные слова посчитаются как ошибки, потому что *удивительно* Наташа не поняла, что эти обрубки когда-то были глаголами

In [181]:
for i in range(64, 84):
    natasha_corpus[i] = natasha_corpus[i+2]
    
for i in range(84, 245):
    natasha_corpus[i] = natasha_corpus[i+4]
    
natasha_corpus.pop(245)
natasha_corpus.pop(246)
natasha_corpus.pop(247)
natasha_corpus.pop(248)

natasha_corpus

{0: {'word': 'Я', 'tag': 'PRO'},
 1: {'word': 'вырос', 'tag': 'V'},
 2: {'word': 'в', 'tag': 'PR'},
 3: {'word': 'то', 'tag': 'PRO'},
 4: {'word': 'время', 'tag': 'S'},
 5: {'word': 'когда', 'tag': 'CONJ'},
 6: {'word': 'бутер', 'tag': 'S'},
 7: {'word': 'еще', 'tag': 'ADV'},
 8: {'word': 'был', 'tag': 'V'},
 9: {'word': 'бутербродом', 'tag': 'V'},
 10: {'word': 'Когда', 'tag': 'CONJ'},
 11: {'word': 'видишь', 'tag': 'V'},
 12: {'word': 'алфавит', 'tag': 'S'},
 13: {'word': 'всё', 'tag': 'PRO'},
 14: {'word': 'время', 'tag': 'S'},
 15: {'word': 'ищешь', 'tag': 'V'},
 16: {'word': 'ответ', 'tag': 'S'},
 17: {'word': 'на', 'tag': 'PR'},
 18: {'word': 'вопрос', 'tag': 'S'},
 19: {'word': 'ГДЕ', 'tag': 'S'},
 20: {'word': 'ЁЖ', 'tag': 'ADV'},
 21: {'word': 'Только', 'tag': 'ADV'},
 22: {'word': 'в', 'tag': 'PR'},
 23: {'word': 'русском', 'tag': 'A'},
 24: {'word': 'языке', 'tag': 'S'},
 25: {'word': 'можно', 'tag': 'ADV'},
 26: {'word': 'составить', 'tag': 'V'},
 27: {'word': 'предложение'

Теперь и у Наташи все хорошо
# 3. Accuracy
Sklearn_accuracy не пройдет, потому что он проводит сравнение на предмет полного соответствия, а нам необходимо рассмотреть вариативность в теге  
Формула оценки

In [185]:
def accuracy(x_corpus):
    right = 0
    wrong = 0
    
    for num, val in corpus.items():
        
        if x_corpus[num]['tag'] in val['tags']: 
            right += 1
        
        else:
            wrong += 1
            
    accuracy = right / (right+wrong)
    
    return accuracy

**Mystem**

In [187]:
accuracy(mystem_corpus)

0.9020408163265307

**Pymorphy**

In [188]:
accuracy(pymorphy_corpus)

0.8489795918367347

**Natasha**

In [186]:
accuracy(natasha_corpus)

0.8448979591836735

Лучше всех справился mystem!!!
# 4. Чанкер
### 4.1. Шаблоны
В качестве шаблонов я буду использовать отрицания с различными частями речи, потому что оно меняет тональность на противоположную, а значит это наиболее важно для ее оценки
1) Не + V (не берите)  
2) Нет + S (нет вкуса)  
3) Не + ADJ (не очень)
### 4.2. Сам чанкер
Возвращает отрицательное число, равное по модулю количеству биграмм, которые соответсвуют шаблонам

In [234]:
def chunker(text):
    res = 0
    ana = []
    old_ana = m.analyze(text)
    for word in old_ana:
        if 'analysis' in word:
            ana.append(word)
    for i in range(len(ana)):
        if ana[i]['text'] in ['Не', 'не', 'НЕ'] and i < len(ana) - 1:
            if 'analysis' in ana[i+1]:
                if ana[i+1]['analysis'] != []:
                    pos = ana[i+1]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                    if mystem(pos) in ['V', 'ADV']:
                        res -= 1
                        
        if ana[i]['text'] in ['Нет', 'нет', 'НЕТ'] and i < len(ana) - 1:
            if 'analysis' in ana[i+1]:
                if ana[i+1]['analysis'] != []:
                    pos = ana[i+1]['analysis'][0]['gr'].split('=')[0].split(',')[0]
                    if mystem(pos) in ['S']:
                        res -= 1
                        
    return res

In [235]:
print(chunker('не люблю нет вкуса Не очень нет хочу'))

-3


### 4.3. Улучшенная функция тональности

In [236]:
def tone(test, posx, negx, activate_chunker=False):
    y_pred = []
    
    for text in test:
        val = 0
        tokenized_text = tokenize_string(text)
        
        for token in tokenized_text:
            
            if token in posx:
                val += 1
            
            if token in negx: 
                val -= 1
        
        if val >= 0: 
            y_pred.append(1)
        
        if val < 0:
            y_pred.append(-1)
        
        if activate_chunker:
            val += chunker(text)
        
            
    return y_pred

### 4.4. Подготовка данных
*Кусочек кода, который наконец-то скачал мне набор отзывов на компьютер, чего я почему-то не сделала раньше*

In [189]:
vine_page = [] 

for i in range(1, 107):
    req = 'https://otziv-otziv.ru/katalog/vino/?page=' + str(i)
    response = requests.get(req)
    
    if (response.status_code == 200):
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.findAll('div', {'class': 'content'})
        
        for block in page:
            vine_page.append(block.find('a')['href'])
            
    else:
        print(i, ' error')

In [191]:
len(vine_page)

2332

In [192]:
pos_global = []
neg_global= []

for i in vine_page:
    req = 'https://otziv-otziv.ru' + i
    response = requests.get(req)
    
    if (response.status_code == 200):
        soup = BeautifulSoup(response.text, 'html.parser')
        review = soup.findAll('div', {'class': 'container-reviews'})

        for block in review:
            rating = int(block.find('div', {'class': 'stars-container'}).get('title')[-1])    
            text = block.get_text()[24:]
    
            if rating > 3:
                pos_global.append(text)
                
            elif rating < 3:
                neg_global.append(text)
     
    else:
        print('error')

In [193]:
print('Положительных отзывов: ', len(pos_global), '\nОтрицательных отзывов:', len(neg_global))

Положительных отзывов:  4005 
Отрицательных отзывов: 746


In [194]:
import numpy as np
import random

In [195]:
def here(posx, negx):

    if len(posx) > len(negx):
        a = random.choices(np.array(posx), k=min(len(posx), len(negx)))
        b = negx
    
    elif len(posx) < len(negx):
        a = posx
        b = random.choices(np.array(negx), k=min(len(posx), len(negx)))
    
    else:
        a = posx
        b = negx
        
    return a, b

In [196]:
pos, neg = here(pos_global, neg_global)
print('Положительных отзывов: ', len(pos), '\nОтрицательных отзывов:', len(neg))

Положительных отзывов:  746 
Отрицательных отзывов: 746


In [200]:
dct = {}
dct['positive'] = pos
dct['negative'] = neg
with open('отзывы.json', 'w', encoding='utf-8') as f:
    json.dump(dct, f, ensure_ascii=False, indent='\t')

*Конец кусочка кода, который надо было сделать раньше*  
Скачиваю отзывы

In [201]:
with open('отзывы.json', 'r', encoding='utf-8') as f:
    reviews = json.load(f)

In [238]:
reviews['positive'] = pos
reviews['negative'] = neg

['\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nКомментарий: \n\nНедостатки:\nНе замечено\n\nОтличное вино, очень нравится\r\n\r\n                                ',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nКомментарий: Очень не плохо!\nНо для Чили, малова-то...\n"Столовое", наверное больше подходтит, питкое, вкусное, аромат есть, но при этом не тянет(!)\nМожно купить и пить, если не разбираетесь в винах НС (Новый свет), потому что, стоит своих денег и ОооОчень качественный продукт!\r\n\r\n                                ',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nКомментарий: Достоинства:\nЦена, вкус\n\n\nНедостатки:\nЧувствуется суррогат, красители и добавки\r\n\r\n                                ',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nКомментарий: Отличное вино\r\n\r\n                                ',
 '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nДостоинства:Вкусное, не дорогоеНедостатки:НетКомментарий:Вино ароматное

Подготавливаю, как в первом домашнем задании

In [239]:
sample = []
r = round(min(len(pos), len(neg))/10)

if r%2 == 1:
    r -= 1
    
for i in pos[0: r]:
    sample.append(i)

for i in neg[0: r]:
    sample.append(i)
    
del neg[0: r]
del pos[0: r] 

In [240]:
y = []

for i in range(0, r):
    y.append(1)

for i in range(0, r):
    y.append(-1)

In [242]:
def tokenize_list(lst):
    all_words = []
    
    for comment in lst:
        tokens = nltk.word_tokenize(comment)
        words = [word.lower() for word in tokens if word.isalpha()]
        all_words.extend(lemmatize(words))
    
    return all_words

def tokenize_string(strg):
    tokens = nltk.word_tokenize(strg)
    words = [word.lower() for word in tokens if word.isalpha()]
    lemmas = lemmatize(words)
    
    return lemmas

def lemmatize(lst):
    res = []
    
    for word in lst:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
        
    return res

In [243]:
tokenized_neg = tokenize_list(neg)
tokenized_pos = tokenize_list(pos)

In [244]:
from collections import Counter

In [245]:
counted_neg = Counter(tokenized_neg)
counted_pos = Counter(tokenized_pos)

In [246]:
def frequently(dct, n):
    words = np.array([i for i in dct.keys()])
    frequency = np.array([i for i in dct.values()])
    res = set(words[frequency > n])

    return res

In [247]:
unique_neg = frequently(counted_neg, 2).difference(frequently(counted_pos, 2))
unique_pos = frequently(counted_pos, 2).difference(frequently(counted_neg, 2))

### 4.5. Сравнение

In [248]:
from sklearn.metrics import accuracy_score

**Без чанкера**

In [249]:
accuracy_score(y, tone(sample, unique_pos, unique_neg))

0.7297297297297297

**С чанкером**

In [ ]:
accuracy_score(y, tone(sample, unique_pos, unique_neg, activate_chunker=True))

Достаточно большая проблема состоит в том, что mystem работает долго, а текстов, хоть и небольших, у меня практически полторы тысячи, поэтому я загружу тетрадку без этого числа, если когда-нибудь получится досчитать, то я напишу это число в комментарий к домашнему заданию